# 01 · Data Overview & QC checks
Load water quality data, thresholds, and QC samples. Perform basic sanity checks and flag issues.

In [ ]:

import pandas as pd
from pathlib import Path

DATA_DIR = Path("../data")
df = pd.read_csv(DATA_DIR / "water_quality_samples.csv", parse_dates=["timestamp"])
thr = pd.read_csv(DATA_DIR / "lab_thresholds.csv")
qc = pd.read_csv(DATA_DIR / "qc_blanks_duplicates.csv")

df.head()


## Basic completeness & ranges

In [ ]:

import numpy as np

summary = df.describe(include='all')
nulls = df.isnull().sum().sort_values(ascending=False)

summary, nulls.head(20)


## Rule-based flags vs thresholds

In [ ]:

def flag_param(series, min_ok, max_ok):
    return (~series.between(min_ok, max_ok)).astype(int)

flags = pd.DataFrame({"timestamp": df["timestamp"], "station_id": df["station_id"]})
for _, row in thr.iterrows():
    param = row["parameter"]
    if param in df.columns:
        flags[param+"_flag"] = flag_param(df[param], row["min_ok"], row["max_ok"])

flags.head()


## QC: Relative Percent Difference (RPD) for duplicates

In [ ]:

dups = qc[qc["type"].isin(["PRIMARY","DUPLICATE"])].copy()
paired = dups.pivot_table(index="sample_id", columns="type", values=["nitrate_mgL","phosphate_mgL","ammonia_mgL"], aggfunc="first")
paired.columns = ["_".join(c).lower() for c in paired.columns.to_flat_index()]
paired = paired.dropna()

def rpd(a,b):
    return (abs(a-b) / ((a+b)/2))*100

paired["RPD_nitrate"] = rpd(paired["nitrate_mgL_primary"], paired["nitrate_mgL_duplicate"])
paired["RPD_phosphate"] = rpd(paired["phosphate_mgL_primary"], paired["phosphate_mgL_duplicate"])
paired["RPD_ammonia"] = rpd(paired["ammonia_mgL_primary"], paired["ammonia_mgL_duplicate"])

paired[["RPD_nitrate","RPD_phosphate","RPD_ammonia"]].describe()


## QC: Method blanks & surrogate recovery

In [ ]:

blanks = qc[qc["type"]=="METHOD_BLANK"].copy()
recoveries = qc.loc[qc["surrogate_recovery_pct"].notna(), ["sample_id","surrogate_recovery_pct"]]

blanks.describe(), recoveries.describe()
